In [2]:
import wobble
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = wobble.Data('hip54287_e2ds.hdf5', filepath='data/', orders=[56])

In [ ]:
model = wobble.Model(data)
model.add_star('hip54287b')
model.add_telluric('tellurics')
print(model)

Inject some noise & try to get back to the pipeline values by optimizing:

In [ ]:
import tensorflow as tf
session = wobble.get_session()
session.run(tf.global_variables_initializer())
pipeline_rvs = session.run(model.components[0].rvs_block[0])
model.components[0].rvs_block[0].load(pipeline_rvs + np.random.normal(0., 100., data.N), session)

In [ ]:
star_rvs = session.run(model.components[0].rvs_block[0])
initial_rvs = np.copy(star_rvs)
plt.scatter(data.pipeline_rvs, star_rvs - data.pipeline_rvs)

In [ ]:
%%time
niter = 100
nll_history, rvs_history, model_history, chis_history = wobble.optimize_order(model, data, 0, niter=niter, output_history=True)

In [ ]:
plt.scatter(np.arange(len(nll_history)), nll_history)
ax = plt.gca()
ax.set_yscale('log')

In [ ]:
for c in model.components:
    model_xs = session.run(c.model_xs[0])
    model_ys = session.run(c.model_ys[0])
    plt.plot(np.exp(model_xs), np.exp(model_ys))

In [ ]:
plt.plot(np.exp(model_xs), np.exp(model_ys))
plt.xlim([5899, 5905])
plt.xlabel('Wavelength (A)')
plt.title('Telluric model, order 56 zoom')

In [ ]:
for c in [model.components[0]]:
    model_xs = session.run(c.model_xs[0])
    model_ys = session.run(c.model_ys[0])
plt.plot(np.exp(model_xs), np.exp(model_ys))
plt.xlim([5899, 5905])
plt.xlabel('Wavelength (A)')
plt.title('Star model, order 56 zoom')

In [ ]:
star_rvs = session.run(model.components[0].rvs_block[0])
plt.scatter(data.pipeline_rvs, initial_rvs - data.pipeline_rvs, color='k', alpha=0.6)
plt.scatter(data.pipeline_rvs, star_rvs - data.pipeline_rvs, color='r', alpha=0.6)

In [ ]:
t_rvs = session.run(model.components[1].rvs_block[0])
plt.scatter(data.pipeline_rvs, t_rvs, color='k', alpha=0.6)

In [ ]:
from IPython.display import HTML
ani = wobble.plot_rv_history(data, rvs_history, niter, niter, ylims=[-100, 100])
HTML(ani.to_html5_video())

In [ ]:
data_xs = session.run(data.xs[0])
ani = wobble.plot_chis_history(0, data_xs, chis_history, niter, 50, ylims=(-1.e2, 1.e2))
HTML(ani.to_html5_video())

In [ ]:
model_xs = session.run(model.components[0].model_xs[0])
ani = wobble.plot_model_history(model_xs, model_history, niter, 50)
HTML(ani.to_html5_video())

Gradient checks:

In [ ]:
tf_grad = session.run(model.components[0].gradients_rvs)[0]

In [ ]:
r = 0
rvs = tf.placeholder(tf.float64, (75,))
synth = model.components[1].shift_and_interp(r, data.xs[r], model.components[1].rvs_block[r])
synth += model.components[0].shift_and_interp(r, data.xs[r], rvs)
nll = 0.5*tf.reduce_sum(tf.square(data.ys[r] - synth) * data.ivars[r])

In [ ]:
num_grads = np.zeros_like(tf_grad)
rvs0 = session.run(model.components[0].rvs_block[0])
nll_0 = session.run(nll, feed_dict={rvs:rvs0})
d_rvs = 0.001
for n in range(75):
    rvs1 = np.copy(rvs0)
    rvs1[n] += d_rvs
    nll_1 = session.run(nll, feed_dict={rvs:rvs1})
    num_grads[n] = np.copy((nll_1 - nll_0)/d_rvs)
print("Fractional difference between numerical gradients & TF gradient at each RV_star:")
print((num_grads - tf_grad) / np.median(np.abs(tf_grad)))

In [ ]:
from wobble.interp import interp
model_len = 6815

model_ys = tf.placeholder(tf.float64, (model_len,))
shifted_xs = data.xs[r] + tf.log(wobble.doppler((model.components[0].rvs_block[r])[:, None]))
synth = interp(shifted_xs, model.components[0].model_xs[r], model_ys) 
synth += model.components[1].shift_and_interp(r, data.xs[r], model.components[1].rvs_block[r])
nll = 0.5*tf.reduce_sum(tf.square(data.ys[r] - synth) * data.ivars[r])

In [ ]:
tf_grad = session.run(model.components[0].gradients_model)[0]
dense_grad = np.zeros(model_len)
for i in range(model_len):
    dense_grad[i] = np.sum(tf_grad.values[tf_grad.indices == i])

In [ ]:
model0 = session.run(model.components[0].model_ys[0])
nll_0 = session.run(nll, feed_dict={model_ys:model0})
d_model = 0.001
diff = []
for n in np.random.randint(0, model_len, size=20):
    model1 = np.copy(model0)
    model1[n] += d_model
    nll_1 = session.run(nll, feed_dict={model_ys:model1})
    diff.append((nll_1 - nll_0)/d_rvs - dense_grad[n])
print("Fractional difference between numerical gradients & TF gradient for random model pixels:")
print(diff / np.median(np.abs(dense_grad)))

Now let's try all orders!

In [3]:
data = wobble.Data('hip54287_e2ds.hdf5', filepath='data/', orders=np.arange(72))
model = wobble.Model(data)
model.add_star('hip54287b')
model.add_telluric('tellurics')
print(model)

Model consisting of the following components: ['hip54287b', 'tellurics']


In [4]:
plot_dir = 'results/plots/'
niter = 60
for r in range(data.R):
    nll_history, rvs_history, model_history, chis_history = wobble.optimize_order(model, data, r, 
                niter=niter, output_history=True) 
    plt.scatter(np.arange(len(nll_history)), nll_history)
    ax = plt.gca()
    ax.set_yscale('log')
    plt.savefig(plot_dir+'nll_order{0}.png'.format(r))   
    plt.show()
    rvs_ani = wobble.plot_rv_history(data, rvs_history, niter, 50, xlims=[-20000, 20000], ylims=[-50, 50])
    rvs_ani.save(plot_dir+'rvs_order{0}.mp4'.format(r), fps=30, extra_args=['-vcodec', 'libx264'])
    print('RVs animation saved')
    session = wobble.get_session()
    model_xs = session.run(model.components[0].model_xs[r])
    model_ani = wobble.plot_model_history(model_xs, model_history, niter, 50)
    model_ani.save(plot_dir+'model_order{0}.mp4'.format(r), fps=30, extra_args=['-vcodec', 'libx264'])
    print('model animation saved')
    data_xs = session.run(data.xs[r])
    chis_ani = wobble.plot_chis_history(0, data_xs, chis_history, niter, 50)
    chis_ani.save(plot_dir+'chis_order{0}_epoch0.mp4'.format(r), fps=30, extra_args=['-vcodec', 'libx264'])
    print('chis animation saved')

initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
initializing star model...
i

AttributeError: 'NoneType' object has no attribute 'shape'